---

## Rebalancing Allocation

To rebalance allocations of subportfolios (a subportfolio is a collection of strategies), we must get the performance of each strategy for each period of time and applying it for the next time period, when using walk forward optimization in the testing module

In [73]:

import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid

# 1. Load historical data
def load_data(file_path):
    data = pd.read_csv(file_path, parse_dates=["date"], index_col="date")
    return data

# 2. Define a sample trading strategy
def trading_strategy(data, short_window, long_window):
    data["SMA_Short"] = data["close"].rolling(window=short_window).mean()
    data["SMA_Long"] = data["close"].rolling(window=long_window).mean()
    data["Signal"] = np.where(data["SMA_Short"] > data["SMA_Long"], 1, -1)  # 1 for Buy, -1 for Sell
    data["Strategy_Return"] = data["Signal"].shift(1) * data["return"]
    return data

# 3. Split data into in-sample and out-of-sample periods
def split_data(data, train_size, test_size, step_size):
    start = 0
    while start + train_size + test_size <= len(data):
        train = data.iloc[start:start + train_size]
        test = data.iloc[start + train_size:start + train_size + test_size]
        yield train, test
        start += step_size

# 4. Optimize parameters on in-sample data
def optimize_parameters(train_data, param_grid):
    best_params = None
    best_performance = -np.inf
    for params in ParameterGrid(param_grid):
        result = trading_strategy(train_data.copy(), **params)
        performance = result['creturns'].iloc[-1]
        if performance > best_performance:
            best_performance = performance
            best_params = params
    return best_params

# 5. Test optimized parameters on out-of-sample data
def test_strategy(test_data, best_params):
    result = trading_strategy(test_data.copy(), **best_params)
    return result["creturns"].iloc

# 6. Walk-forward optimization loop
def walk_forward_optimization(data, train_size, test_size, step_size, param_grid):
    results = []
    for train, test in split_data(data, train_size, test_size, step_size):
        # Optimize on training data
        best_params = optimize_parameters(train, param_grid)
        # Test on out-of-sample data
        performance = test_strategy(test, best_params)
        results.append(performance)
    return results

# Example Usage
if __name__ == "__main__":
    # Load data
    data = load_data("data.csv")
    data = data[data['coin'] == 'BTCUSDT']
    data["return"] = data["close"].pct_change()

    # Parameter grid for optimization
    param_grid = {"short_window": range(5, 21, 5), "long_window": range(20, 101, 20)}

    # Walk-forward optimization
    train_size = 250  # Number of training days
    test_size = 50    # Number of testing days
    step_size = 50    # Step size for walk forward
    results = walk_forward_optimization(data.dropna(), train_size, test_size, step_size, param_grid)

    # Aggregate and analyze results
    print(f"Average out-of-sample performance: {np.mean(results):.2f}")

Average out-of-sample performance: 0.98


In [74]:
results

[1.004935817887272,
 0.9934966803004952,
 0.99729371683214,
 0.9842931986477768,
 0.9804281224610204,
 0.9292170990009944,
 0.9392749609595998,
 0.99121817586391,
 1.0222779252755083]